In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/AI_Chip_Contest')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -qU datasets huggingface_hub
!pip install -qU transformers huggingface_hub safetensors accelerate


from huggingface_hub import login
login(token="your_api_key")

import transformers
transformers.logging.set_verbosity_error()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.6 MB/s eta 0:00:00




-----

### **LLM 프루닝 기술 비교: SparseGPT vs. Wanda vs. FLAP**

세 기술 모두 재학습이 필요 없는 **원샷(One-shot) 프루닝** 방식이지만, 중요도를 판단하고 가중치를 처리하는 철학에서 큰 차이를 보입니다.

### **한눈에 보는 비교**

| 구분 | SparseGPT (정교한 보상) | Wanda (빠른 판단) | FLAP (구조적 설계) |
| :--- | :--- | :--- | :--- |
| **핵심 아이디어** | 2차 정보(헤시안 행렬)를 이용해\<br\>오차를 계산하고 남은 가중치 보정 | 가중치와 입력 활성화 크기의 곱\<br\>`|W| x ||X||` | 출력 변동성을 기반으로\<br\>구조적 중요도 판단 |
| **프루닝 종류** | 비구조적 / 구조적 | 비구조적 / 구조적 | **구조적(Structured)** |
| **가중치 처리**| **자르고 보상하기** | **자르기만 함** | 자르고 일부 보상(Bias) |
| **주요 장점** | 매우 높은 정확도 유지 | 압도적인 속도와 단순함 | **실제 하드웨어 가속**에 유리 |
| **주요 단점** | 계산 비용이 높고 복잡함 | SparseGPT보다 정확도 하락 가능성 | 비구조적 프루닝보다 유연성 낮음 |

-----

### **1. SparseGPT: 정교한 외과수술 🩺**

  * **특징**:
      * **2차 정보(헤시안) 활용**: 가중치 간의 상호작용까지 고려하여 제거할 가중치를 선택합니다.
      * **오차 보상**: 가중치를 하나 제거할 때마다 그로 인한 출력 오차를 최소화하도록, **남아있는 다른 가중치들의 값을 즉시 업데이트**합니다. '자르고 보상하는(prune and update)' 방식의 대표주자입니다.
  * **장점**: 프루닝 후에도 원본 모델의 성능을 매우 높은 수준으로 보존합니다.
  * **단점**: 헤시안 행렬을 계산하고 역행렬을 구하는 등 연산 과정이 복잡하여 다른 원샷 기법들보다 시간이 더 오래 걸립니다.

-----

### **2. Wanda: 빠르고 실용적인 판단 ⚡**

  * **특징**:
      * **단순한 중요도 공식**: `|가중치 크기| × ||입력 활성화 크기||` 라는 직관적인 공식을 사용합니다.
      * **업데이트 없음**: 가중치를 제거한 후, 남은 가중치를 보정하는 과정이 전혀 없습니다. '자르기만 하는(prune-only)' 방식입니다.
  * **장점**: 구현이 매우 쉽고 계산 비용이 압도적으로 낮아 **매우 빠릅니다.** 그럼에도 불구하고 LLM에서 높은 성능을 보여 실용성이 뛰어납니다.
  * **단점**: 가중치를 보정하지 않으므로, SparseGPT에 비해 높은 희소성(sparsity)에서 성능 하락폭이 더 클 수 있습니다.

-----

### **3. FLAP: 하드웨어를 고려한 구조 설계 🏗️**

  * **특징**:
      * **구조 단위 제거**: 개별 가중치가 아닌, **가중치 행렬의 행/열이나 어텐션 헤드 같은 구조적인 덩어리**를 통째로 제거합니다.
      * **변동성 기반 판단**: 특정 구조를 제거했을 때 모델의 최종 출력이 얼마나 흔들리는지(fluctuation)를 측정하여 중요도를 판단합니다.
      * **적응형 희소성**: 모델의 레이어마다 중요도를 다르게 판단하여, 덜 중요한 레이어는 더 많이 제거하는 등 프루닝 비율을 동적으로 조절합니다.
  * **장점**: 구조 단위로 제거하므로 **실제 하드웨어에서 연산량 감소와 추론 속도 향상 효과**를 직접적으로 볼 수 있습니다.
  * **단점**: 비구조적 프루닝만큼 세밀하게 가중치를 제거하지 못하므로, 동일한 희소성 비율에서 정확도 하락이 더 클 수 있습니다.



In [ ]:
## sparseGPT
%cd sparsegpt
!python main.py --model meta-llama/Llama-3.1-8B-Instruct --sparsity 0.3 --save /content/drive/MyDrive/AI_Chip_Contest/models/llama_sparsegpt_sparsity_03

!python main.py --model meta-llama/Llama-3.1-8B-Instruct --sparsity 0.5 --save /content/drive/MyDrive/AI_Chip_Contest/models/llama_sparsegpt_sparsity_05

!python main.py --model meta-llama/Llama-3.1-8B-Instruct --sparsity 0.7 --save /content/drive/MyDrive/AI_Chip_Contest/models/llama_sparsegpt_sparsity_07

In [ ]:
%cd wanda
!python main_wanda.py \
    --model meta-llama/Llama-3.1-8B-Instruct \
    --prune_method wanda \
    --sparsity_ratio 0.3 \
    --nsamples 128 \
    --sparsity_type unstructured \
    --save_model /content/drive/MyDrive/AI_Chip_Contest/models/llama_wanda_sparsity_03

!python main_wanda.py \
    --model meta-llama/Llama-3.1-8B-Instruct \
    --prune_method wanda \
    --sparsity_ratio 0.5 \
    --nsamples 128 \
    --sparsity_type unstructured \
    --save_model /content/drive/MyDrive/AI_Chip_Contest/models/llama_wanda_sparsity_05

!python main_wanda.py \
    --model meta-llama/Llama-3.1-8B-Instruct \
    --prune_method wanda \
    --sparsity_ratio 0.7 \
    --nsamples 128 \
    --sparsity_type unstructured \
    --save_model /content/drive/MyDrive/AI_Chip_Contest/models/llama_wanda_sparsity_07

In [ ]:
%cd FLAP
!python main.py \
    --model meta-llama/Llama-3.1-8B-Instruct \
    --prune_method flap \
    --pruning_ratio 0.3 \
    --remove_heads -1 \
    --unstr \
    --metrics WIFV \
    --structure AL-AM \
    --nsamples 256 \
    --save_model "/content/drive/MyDrive/AI_Chip_Contest/models/llama_flap_sparsity_03"
    #--eval \

!python main.py \
    --model meta-llama/Llama-3.1-8B-Instruct \
    --prune_method flap \
    --pruning_ratio 0.5 \
    --remove_heads -1 \
    --unstr \
    --metrics WIFV \
    --structure AL-AM \
    --nsamples 256 \
    --save_model "/content/drive/MyDrive/AI_Chip_Contest/models/llama_flap_sparsity_05"
    #--eval \

!python main.py \
    --model meta-llama/Llama-3.1-8B-Instruct \
    --prune_method flap \
    --pruning_ratio 0.7 \
    --remove_heads -1 \
    --unstr \
    --metrics WIFV \
    --structure AL-AM \
    --nsamples 256 \
    --save_model "/content/drive/MyDrive/AI_Chip_Contest/models/llama_flap_sparsity_07"
    #--eval \


In [ ]:
!python eval_pruning_model.py --model_id "meta-llama/Llama-3.1-8B-Instruct" --mmlu_samples 1000

In [ ]:
!python eval_pruning_model.py --model_id "./models/llama_sparsegpt_sparsity_03" --mmlu_samples 1000
!python eval_pruning_model.py --model_id "./models/llama_sparsegpt_sparsity_05" --mmlu_samples 1000
!python eval_pruning_model.py --model_id "./models/llama_sparsegpt_sparsity_07" --mmlu_samples 1000

In [ ]:
!python eval_pruning_model.py --model_id "./models/llama_wanda_sparsity_03" --mmlu_samples 1000
!python eval_pruning_model.py --model_id "./models/llama_wanda_sparsity_05" --mmlu_samples 1000
!python eval_pruning_model.py --model_id "./models/llama_wanda_sparsity_07" --mmlu_samples 1000

In [ ]:
!python eval_pruning_model.py --model_id "./models/llama_flap_sparsity_03" --mmlu_samples 1000
!python eval_pruning_model.py --model_id "./models/llama_flap_sparsity_05" --mmlu_samples 1000
!python eval_pruning_model.py --model_id "./models/llama_flap_sparsity_07" --mmlu_samples 1000